# PCHorvath2013

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.PCHorvath2013)

class PCHorvath2013(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        """
        Applies an anti-logarithmic linear transformation to a PyTorch tensor.
        """
        adult_age = 20

        # Create a mask for negative and non-negative values
        mask_negative = x < 0
        mask_non_negative = ~mask_negative

        # Initialize the result tensor
        age_tensor = torch.empty_like(x)

        # Exponential transformation for negative values
        age_tensor[mask_negative] = (1 + adult_age) * torch.exp(x[mask_negative]) - 1

        # Linear transformation for non-negative values
        age_tensor[mask_non_negative] = (1 + adult_age) * x[
            mask_non_negative
        ] + adult_age

        return age_tensor



In [3]:
model = pya.models.PCHorvath2013()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'pchorvath2013'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Higgins-Chen, Albert T., et al. \"A computational solution for bolstering reliability of epigenetic clocks: Implications for clinical trials and longitudinal tracking.\" Nature aging 2.7 (2022): 644-661."
model.metadata["doi"] = "https://doi.org/10.1038/s43587-022-00248-2"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

In [5]:
#download PCClock Rdata file from https://yale.app.box.com/s/kq0b0a7lxckxjvaz7x5n4keaug7tewry
logger = pya.logger.Logger()
url = "https://pyaging.s3.amazonaws.com/supporting_files/CalcAllPCClocks.RData"
dir = "."
pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Downloading data to ./CalcAllPCClocks.RData


|-----------> in progress: 1.0003%

|-----------> in progress: 2.0006%

|-----------> in progress: 3.0009%

|-----------> in progress: 4.0012%

...

|-----------> in progress: 96.0288%

|-----------> in progress: 97.0291%

|-----------> in progress: 98.0294%

|-----------> in progress: 99.0297%

|-----------> in progress: 100.0000%


#### Download from R package

In [6]:
%%writefile download.r

library(dplyr)
library(tibble)
library(tidyr)
library(jsonlite)

load(file = "CalcAllPCClocks.RData")

print(ls(all.names = TRUE))

write_json(CalcPCHorvath1, "CalcPCHorvath1.json", digits = 10)
write_json(CpGs, "PCHorvath1CpGs.json")
write_json(imputeMissingCpGs, "PCHorvath1ReferenceCpGBetas.json", digits = 10)

Writing download.r


In [7]:
os.system("Rscript download.r")

0

## Load features

#### From JSON file

In [8]:
with open('PCHorvath1CpGs.json', 'r') as f:
    model.features = json.load(f)

## Load weights into base model

#### From JSON file

In [9]:
with open('CalcPCHorvath1.json', 'r') as f:
    weights_dict = json.load(f)

weights = torch.tensor(weights_dict['model']).unsqueeze(0).float()
intercept = torch.tensor(weights_dict['intercept']).float()
center = torch.tensor(weights_dict['center']).float()
rotation = torch.tensor(weights_dict['rotation']).float()

#### PC linear model

In [10]:
base_model = pya.models.PCLinearModel(input_dim=len(model.features), pc_dim=rotation.shape[1])

base_model.center.data = center.float()
base_model.rotation.data = rotation.float()
base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

#### From JSON file

In [11]:
with open('PCHorvath1ReferenceCpGBetas.json', 'r') as f:
    reference_feature_values = json.load(f)
model.reference_values = reference_feature_values

## Load preprocess and postprocess objects

In [12]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [13]:
model.postprocess_name = 'anti_log_linear'
model.postprocess_dependencies = None

## Check all clock parameters

In [14]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Higgins-Chen, Albert T., et al. "A computational solution for '
             'bolstering reliability of epigenetic clocks: Implications for '
             'clinical trials and longitudinal tracking." Nature aging 2.7 '
             '(2022): 644-661.',
 'clock_name': 'pchorvath2013',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s43587-022-00248-2',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: [0.82635363384, 0.18898814441, 0.72938889209, 0.8680421375, 0.090353927561, 0.0066895021761, 0.48924643338, 0.87262052546, 0.87955373232, 0.04847264273, 0.0093070979947, 0.16393676218, 0.058440936082, 0.18857484916, 0.58239394253, 0.86564960457, 0.58457176982, 0.82903550669, 0.065646928047, 0.8500055061, 0.79155429878, 0.83499889314, 0

## Basic test

In [15]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[59.4351],
        [58.9732],
        [36.1313],
        [11.8124],
        [37.2571],
        [42.6807],
        [35.9014],
        [45.7016],
        [61.6516],
        [59.3143]], dtype=torch.float64, grad_fn=<IndexPutBackward0>)

## Save torch model

In [16]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [17]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: PCHorvath1CpGs.json
Deleted file: CalcPCHorvath1.json


Deleted file: CalcAllPCClocks.RData
Deleted file: download.r
Deleted file: PCHorvath1ReferenceCpGBetas.json
